In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.layers import LSTM, Dropout, Dense
from tensorflow.keras.models import Sequential

In [2]:
dataset_train = pd.read_csv('Google_Stock_Price_Train.csv')

#keras only takes numpy array
training_set = dataset_train.iloc[:, 1: 2].values

In [3]:
# Feature scaling

from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0, 1))
training_set_scaled = sc.fit_transform(training_set)

In [4]:
# Data structure creation

X_train = []
y_train = []
for i in range(60, len(training_set_scaled)):
    X_train.append(training_set_scaled[i-60: i, 0])
    y_train.append(training_set_scaled[i, 0])
    
X_train, y_train = np.array(X_train), np.array(y_train)

In [5]:
# Data reshaping

X_train = np.reshape(X_train, newshape = (X_train.shape[0], X_train.shape[1], 1))

In [6]:
# Model building

regressor = Sequential()

In [7]:
# Add the 1st LSTM layer with the Dropout layer followed.

regressor.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train.shape[1], 1)))
regressor.add(Dropout(rate = 0.2))

In [8]:
## Add 2nd lstm layer
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(rate = 0.2))

## Add 3rd lstm layer
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(rate = 0.2))

## Add 4th lstm layer
regressor.add(LSTM(units = 50, return_sequences = False))
regressor.add(Dropout(rate = 0.2))

In [9]:
regressor.add(Dense(units = 1))

In [10]:
# Model compiling

regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

In [ ]:
# Model fitting

regressor.fit(x = X_train, y = y_train, batch_size = 32, epochs = 100)

Epoch 1/100
38/38 [==============================] - 10s 91ms/step - loss: 0.0418
Epoch 2/100
38/38 [==============================] - 4s 92ms/step - loss: 0.0065
Epoch 3/100
38/38 [==============================] - 4s 93ms/step - loss: 0.0063
Epoch 4/100
38/38 [==============================] - 3s 91ms/step - loss: 0.0052
Epoch 5/100
38/38 [==============================] - 3s 81ms/step - loss: 0.0056
Epoch 6/100
38/38 [==============================] - 3s 79ms/step - loss: 0.0048
Epoch 7/100
38/38 [==============================] - 3s 85ms/step - loss: 0.0048
Epoch 8/100
38/38 [==============================] - 3s 79ms/step - loss: 0.0044
Epoch 9/100
38/38 [==============================] - 3s 79ms/step - loss: 0.0049
Epoch 10/100
38/38 [==============================] - 4s 110ms/step - loss: 0.0052
Epoch 11/100
38/38 [==============================] - 5s 120ms/step - loss: 0.0042
Epoch 12/100
38/38 [==============================] - 4s 116ms/step - loss: 0.0036
Epoch 13/100
38/38 [=

In [ ]:
dataset_test = pd.read_csv('Google_Stock_Price_Test.csv')
real_stock_price = dataset_test.iloc[:, 1: 2].values

In [ ]:
# Data processing

dataset_total = pd.concat((dataset_train['Open'],dataset_test['Open']), axis = 0)

In [ ]:
inputs = dataset_total[len(dataset_total)-len(dataset_test)- 60: ].values

In [ ]:
inputs = inputs.reshape(-1, 1)

In [ ]:
inputs = sc.transform(inputs)

In [ ]:
X_test = []
for i in range(60, len(inputs)): 
    X_test.append(inputs[i-60: i, 0])

X_test = np.array(X_test)
# Make numpy array as 3D , adding num of indicator
X_test = np.reshape(X_test, newshape = (X_test.shape[0],  
                    X_test.shape[1], 1))

In [ ]:
predicted_stock_price = regressor.predict(X_test)

In [ ]:
predicted_stock_price = sc.inverse_transform(predicted_stock_price)

In [ ]:
plt.plot(real_stock_price, color = 'red', label = 'Real price')
plt.plot(predicted_stock_price, color = 'blue', label = 'Predicted price')
plt.title('Google price prediction')
plt.xlabel('Time')
plt.ylabel('Price')
plt.legend()
plt.show()